In [ ]:
#모듈 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

#그래프 한글깨짐 방지
plt.rcParams["font.family"] = "Malgun Gothic"	

#Pandas 옵션 설정(필요시 실행)
#pd.set_option("display.max_columns", None) #Pandas 모든열 출력 옵션 설정
#pd.set_option("display.max_rows", None) #Pandas 모든행 출력 옵션 설정
#pd.options.display.max_columns = 20 #Pandas 컬럼 출력갯수 변경 옵션 설정

### 데이터 로드 및 전처리

In [ ]:
#데이터 불러오고 확인
df = pd.read_csv("dataset_hcr/hcr.csv", encoding="CP949")	#데이터 불러오기
df.head() #앞 5행 출력

In [ ]:
df.describe() #수치형 데이터 요약

In [ ]:
df.shape #데이터 모양 확인 (113367, 34)

In [ ]:
df.dtypes #변수별 유형 출력

In [ ]:
#수치형 변수만 남기고 범주형 변수들은 제거
#추후 범주형 변수를 one-hot encoding 하여 모델을 다시 만들어 볼 것
numeric_vars = [] #수치형 변수를 저장할 빈 리스트 생성
for col in df.columns: #df 컬럼명으로 반복
    if df[col].dtype!="object": #반복중인 df 컬럼의 데이터 타입이 범주형에 해당하지 않는다면
        numeric_vars.append(col) #컬럼명을 수치형 변수 리스트에 추가
df = df[numeric_vars] #수치형 변수에 해당하는 컬럼만 떼어내 df에 덮어쓰기
df.info()
#아래와 같이 한줄로 간단하게 할 수도 있음
# df = df[[x for x in df.columns if df[x].dtype != "object"]]

In [ ]:
#결측치 확인
df.isnull().sum()

In [ ]:
#결측치 제거
df.dropna(inplace=True)
df.shape #데이터셋 크기 확인 : (113263,25) → 기존 113367행에서 113263행으로 결측치 제거됨

In [ ]:
#탐색적 분석 : 장입온도(target)과 독립변수간의 관계를 산점도로 그려 확인
#KDE(커널밀도추정) 방식으로 지정해 히스토그램은 확률밀도함수를 추정하는 형태로 표현
sns.pairplot(df[["장입온도", "SLAB폭", "SLAB길이", "SLAB중량", "단위내순서"]], diag_kind="kde")

In [ ]:
#독립변수 표준화를 위해 변수별 평균과 표준편차를 계산하고, 추후 활용을 위해 csv로 저장
stats = df.describe() #데이터 요약
stats = stats.transpose() #행열 전치
stats.to_csv("hcr_stats.csv", encoding="CP949") #저장
stats

In [ ]:
#데이터셋 분할 train 80%, test 20%
x_train = df.sample(frac=0.8, random_state=0) #데이터셋 분리 train 80%
x_test = df.drop(x_train.index) #test 데이터셋 생성
y_train = x_train.pop("장입온도") #x_train에서 장입온도 열 정보만 뽑아 y_train에 저장
y_test = x_test.pop("장입온도") #x_test에서 장입온도 열 정보만 뽑아 y_test에 저장

x_train.shape, x_test.shape, y_train.shape, y_test.shape #train, test 데이터셋 확인

In [ ]:
#수치형 데이터셋 표준화 함수 정의
def norm(df):
    result = df.copy()
    for col in df.columns:
        result[col] = (df[col] - stats.loc[col, "mean"]) / stats.loc[col, "std"]
    return result

x_train_normed = norm(x_train) #x_train 표준화
x_test_normed = norm(x_test) #x_test 표준화

### 모델 구성 및 학습

In [ ]:
#네트워크 구조 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=[len(x_train.columns)]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)]
)

model.summary() #네트워크 구조 출력

In [ ]:
#모델 컴파일. 손실함수로 mse, 최적화 함수로 Adam, 평가지표로 mae, mse 사용
optimizer = tf.keras.optimizers.Adam() #tf.keras.optimizers.RMSprop()으로도 변경해서 학습해볼것
model.compile(loss="mse", optimizer=optimizer, metrics=["mae", "mse"]) #손실함수는 MSE, 평가지표는 MAE와 MSE

#학습 및 평가
#train 데이터 중 25%를 validation 데이터로 나눠 학습, val_loss가 20 epochs 동안 향상이 안될때 학습 종료
history= model.fit(x_train_normed, y_train, epochs=500, batch_size=32, validation_split=0.25,
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=20, monitor="val_loss")])

model.evaluate(x_test_normed, y_test, verbose=0) #test 데이터셋으로 모델 평가

In [ ]:
#history 출력
hist = pd.DataFrame(history.history) #history.history에 저장된 값을 데이터프레임으로 변환
hist["epoch"] = history.epoch #hist 데이터프레임에 ‘epoch’열 추가
hist.tail()

In [ ]:
#train, validation 데이터셋의 mae, mse 그래프 출력
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1) #MAE 그래프 출력
plt.xlabel("Epoch")
plt.ylabel("Mean Abs Error")
plt.plot(hist["epoch"], hist["mae"], label="Train Error")
plt.plot(hist["epoch"], hist["val_mae"], label = "Val Error")
plt.legend()

plt.subplot(1, 2, 2) #MSE 그래프 출력
plt.xlabel("Epoch")
plt.ylabel("Mean Square Error")
plt.plot(hist["epoch"], hist["mse"], label="Train Error")
plt.plot(hist["epoch"], hist["val_mse"], label = "Val Error")
plt.legend()
plt.show()

In [ ]:
#테스트 결과 확인
pred = model.predict(x_test_normed)
df_test = pd.DataFrame({
    "실제" : y_test[:10], "예측" : pred[:10, 0], "잔차" : pred[:10, 0] - y_test[:10]
})
df_test

In [ ]:
#산점도로 테스트 결과 확인
plt.scatter(y_test, pred, marker=".")
plt.xlabel("장입온도 실측값")
plt.ylabel("장입온도 예측값")
plt.show()

In [ ]:
#결정계수(r squared) 산출
y = y_test.to_numpy()
ybar = np.sum(y)/len(y)
ssr = np.sum((pred-ybar)**2)
sst =  np.sum((y-ybar)**2)
r_squared =ssr/sst
print("r_squared : {}%".format(round(r_squared*100,2)))

#케라스 모델로 저장
save_path = "hcr_model.h5"
model.save(save_path, include_optimizer=True)

### 모델 활용

In [ ]:
#저장한 모델 활용을 위한 predict 코드
#새로운 데이터에 개발한 모델을 활용하기 위해서는 모델 개발시의 전처리 과정을 똑같이 거쳐야 함
import pandas as pd
import numpy as np
import tensorflow as tf
plt.rcParams["font.family"] = "Malgun Gothic"

df = pd.read_csv("dataset_hcr/hcr_predict.csv", encoding="CP949") #새로운 데이터 불러오기
stats = pd.read_csv("hcr_stats.csv", index_col=0, encoding="CP949") #첫번째 열을 인덱스로 지정

#모델 불러오기
hcr_model = tf.keras.models.load_model("hcr_model.h5")

#수치형 변수들의 컬럼만 뽑아 numeric_vars에 저장
numeric_vars = []
for col in df.columns:
    if df[col].dtypes != "object":
        numeric_vars.append(col)

#데이터 전처리, 결측치 없는 수치형 데이터 추출
df_treatment = df[numeric_vars]
df_treatment

In [ ]:
#데이터셋 표준화
def norm(df):
    result = df.copy()
    for col in df.columns:
        result[col] = (df[col] - stats.loc[col, "mean"]) / stats.loc[col, "std"]
    return result

normed = norm(df_treatment)

#예측
pred = hcr_model.predict(normed)
df["장입온도예측치"] = pred.astype("int") #실수를 정수로 변환
df.to_csv("hcr_result.csv", index=None, encoding="CP949")	#결과 저장

df
#hcr_result.csv 파일 확인시 마지막열에 장입온도예측치가 추가된것을 확인할 수 있음